# <span style="color:darkblue"> Lecture 19 - Connecting to SQL via Python </span>

<font size = "5">

In the past lecture:

- We worked directly in SQL
- Carried out basic dataset operations

In  this lecture we will run SQL from Python

- Combine the best of both!
- Learn about escape characters!

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">

Import libraries

In [6]:
# psycogpg2 helps us process SQL commands to send to the server
# sqlalchemy facilitates establishing a connection with the server

import pandas as pd
from sqlalchemy import create_engine
from   sqlalchemy.engine import URL
from   sqlalchemy import text

<font size = "5">

Connect to SQL server

- In the default instructions we set <br>
the password to "12345" in windows and <br>
no password for Mac
- ADJUST code accordingly! 

In [7]:
# Use the connection details to your server
# These are the default settings.
# "postgresql" is a fixed argument
# If you have a different host,database, username, or password,
# change the corresponding connection details


url_server = URL.create(
    "postgresql",
    host = 'localhost',
    database = 'postgres',
    username = 'postgres',
    port = 5432,
    password  = "")

connection = create_engine(url_server)


# <span style="color:darkblue"> II. Entity Relationship Diagrams </span>

<font size = "5">

United States Congress dataset

- Bills and actions
- US congress members



<font size = "5">

Read datasets into Python

In [8]:
bills_actions       = pd.read_csv("data_raw/bills_actions.csv")
us_congress_members = pd.read_csv("data_raw/us_congress_members.csv")

<font size = "5">

Upload to SQL


In [10]:
bills_actions.to_sql('bills_actions',
               con = connection,
               if_exists='replace',
               index=False)

# Import "races"
us_congress_members.to_sql('us_congress_members', 
             con = connection, 
             if_exists='replace',
             index=False)

811

<font size = "5">

Download from SQL Server

In [11]:
# Recall that "*" indicates that we should select all columns

data  = pd.read_sql(text('SELECT * \
                          FROM bills_actions;'), connection)

display(data)

,congress,bill_number,bill_type,action,main_action,object,member_id
0,116,1029,s,S.Amdt.1274 Amendment SA 1274 proposed by Sena...,senate amendment proposed (on the floor),amendment,858
1,116,1031,s,S.Amdt.2698 Amendment SA 2698 proposed by Sena...,senate amendment proposed (on the floor),amendment,675
2,116,1160,s,S.Amdt.2659 Amendment SA 2659 proposed by Sena...,senate amendment proposed (on the floor),amendment,858
3,116,1199,s,"Committee on Health, Education, Labor, and Pen...",senate committee/subcommittee actions,senate bill,1561
4,116,1208,s,Committee on the Judiciary. Reported by Senato...,senate committee/subcommittee actions,senate bill,1580
...,...,...,...,...,...,...,...
3298,116,9,hr,H.Amdt.172 Amendment (A004) offered by Ms. Kus...,house amendment offered,amendment,36
3299,116,9,hr,H.Amdt.171 Amendment (A003) offered by Ms. Hou...,house amendment offered,amendment,186
3300,116,9,hr,H.Amdt.170 Amendment (A002) offered by Ms. Oma...,house amendment offered,amendment,477
3301,116,9,hr,POSTPONED PROCEEDINGS - At the conclusion of d...,other house amendment actions,amendment,393


<font size = "5">

Try it yourself

Download and display the "us_congress_members" dataset from SQL

In [14]:
# Write your own code

data  = pd.read_sql(text('SELECT full_name, last_name \
                          FROM us_congress_members;'), connection)

display(data)



,full_name,last_name
0,A. Donald McEachin,McEachin
1,Aaron Schock,Schock
2,Abby Finkenauer,Finkenauer
3,Abigail Davis Spanberger,Spanberger
4,Adam H. Putnam,Putnam
...,...,...
1806,Wm. Lacy Clay,Clay
1807,"Wyche, Jr. Fowler",Fowler
1808,"Wyche, Jr. Fowler",Fowler
1809,Yvette D. Clarke,Clarke


# <span style="color:darkblue"> III. Merge Dataset </span>

<font size = "5">

The following is an example of an entity relationship diagram (ERD)



<font size = "5">

<img src="figures/rdb_us_congress.png" alt="drawing" width="650"/>

- Here "member_id" is the PRIMARY KEY of the first dataset
- We can check that with SQL commands

In [15]:
summary = pd.read_sql(text("SELECT COUNT(DISTINCT member_id) AS num_distinct, \
                           COUNT(*) AS num_members   \
                           FROM us_congress_members"),
                      connection)

summary

,num_distinct,num_members
0,1811,1811


<font size = "5">

Merge two datasets

- Similar to python
- FROM is followed by the name of the primary dataset
- LEFT is followed by the name of the secondary dataset
- ON is the id variable used for merging

In [17]:
# Write your own code

example = pd.read_sql(text("SELECT * \
                            FROM bills_actions \
                            LEFT JOIN us_congress_members \
                            ON bills_actions.member_id = us_congress_members.member_id ;"),
                      connection)

example


,congress,bill_number,bill_type,action,main_action,object,member_id,member_id,full_name,last_name,member_title,state,party_name,chamber
0,116,1029,s,S.Amdt.1274 Amendment SA 1274 proposed by Sena...,senate amendment proposed (on the floor),amendment,858,858,Mitch McConnell,McConnell,Senator,Kentucky,Republican,Senate
1,116,1031,s,S.Amdt.2698 Amendment SA 2698 proposed by Sena...,senate amendment proposed (on the floor),amendment,675,675,Josh Hawley,Hawley,Senator,Missouri,Republican,Senate
2,116,1160,s,S.Amdt.2659 Amendment SA 2659 proposed by Sena...,senate amendment proposed (on the floor),amendment,858,858,Mitch McConnell,McConnell,Senator,Kentucky,Republican,Senate
3,116,1199,s,"Committee on Health, Education, Labor, and Pen...",senate committee/subcommittee actions,senate bill,1561,1561,Lamar Alexander,Alexander,Senator,Tennessee,Republican,Senate
4,116,1208,s,Committee on the Judiciary. Reported by Senato...,senate committee/subcommittee actions,senate bill,1580,1580,Lindsey Graham,Graham,Senator,South Carolina,Republican,Senate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3298,116,9,hr,H.Amdt.172 Amendment (A004) offered by Ms. Kus...,house amendment offered,amendment,36,36,Ann M. Kuster,Kuster,Representative,New Hampshire,Democratic,House
3299,116,9,hr,H.Amdt.171 Amendment (A003) offered by Ms. Hou...,house amendment offered,amendment,186,186,Chrissy Houlahan,Houlahan,Representative,Pennsylvania,Democratic,House
3300,116,9,hr,H.Amdt.170 Amendment (A002) offered by Ms. Oma...,house amendment offered,amendment,477,477,Ilhan Omar,Omar,Representative,Minnesota,Democratic,House
3301,116,9,hr,POSTPONED PROCEEDINGS - At the conclusion of d...,other house amendment actions,amendment,393,393,"Frank, Jr. Pallone",Pallone,Representative,New Jersey,Democratic,House


# <span style="color:darkblue"> IV. (Optional) Additional Resources </span>


<font size = "5">

SQL in 100 seconds:

https://www.youtube.com/watch?v=zsjvFFKOm3c

Overview of SQL:

https://www.youtube.com/watch?v=27axs9dO7AE

Introduction to databases:

https://www.youtube.com/watch?v=wR0jg0eQsZA

Introduction to Entity Relationship Diagrams

https://www.youtube.com/watch?v=wR0jg0eQsZA


